In [1]:
%load_ext autoreload

In [2]:
import sys
sys.path.append("/home/aakash/College/GSoC/neural-extraction-framework/GSoC23")

In [3]:
import pandas as pd
entities = pd.read_csv("./entities_from_abstracts.csv")

In [4]:
# I won't be considering DATE and CARDINAL, ORDINAL, QUANTITY, PERCENT will look at them separately
selected_entities = entities[~entities['entity_group'].isin(
    ['CARDINAL', 'ORDINAL', 'QUANTITY', 'PERCENT', 'DATE'])]

In [49]:
# I won't be considering DATE and CARDINAL, ORDINAL, QUANTITY, PERCENT will look at them separately
selected_entities = entities[~entities['entity_group'].isin(
    ['CARDINAL', 'ORDINAL', 'QUANTITY', 'PERCENT', 'DATE'])]

In [5]:
selected_entities

,Unnamed: 0,entity_group,word,start,end
0,0,PERSON,Lionel Andrés Messi,0,19
1,1,NORP,Spanh,21,28
3,3,PERSON,Leo Messi,103,112
4,4,NORP,Argente,120,129
5,5,ORG,Ligue 1,181,188
...,...,...,...,...,...
275,15,LOC,Europe,602,608
276,16,EVENT,World War II,691,703
281,21,ORG,Project Drawdown's,1510,1528
284,24,GPE,Cha,2592,2597


In [29]:
unique_entities = selected_entities.drop_duplicates()
len(unique_entities), len(selected_entities)

(164, 164)

In [39]:
entity_and_link = {}
%autoreload
from EntityLinking.ELmethods import (
    EL_DBpedia_lookup, 
    EL_DBpedia_spotlight,
    EL_redis_db
    )

In [40]:
entity_and_link

{}

In [7]:
import spacy_dbpedia_spotlight
nlp = spacy_dbpedia_spotlight.create('en')

/home/aakash/miniconda3/envs/gsoc/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
for e, grp in zip(selected_entities['word'], selected_entities['entity_group']):
    ky = e+"_"+grp
    entity_and_link[ky]={}
    entity_and_link[ky]['candidates']=[]

In [42]:
entity_and_link

{'Lionel Andrés Messi_PERSON': {'candidates': []},
 'Spanh_NORP': {'candidates': []},
 'Leo Messi_PERSON': {'candidates': []},
 'Argente_NORP': {'candidates': []},
 'Ligue 1_ORG': {'candidates': []},
 'Par Sat-Germa_ORG': {'candidates': []},
 'Argenta_GPE': {'candidates': []},
 'Messi_PERSON': {'candidates': []},
 "Ballon d'Or awards_WORK_OF_ART": {'candidates': []},
 'European Golden Shoes_WORK_OF_ART': {'candidates': []},
 "Ballon d'Or Dream Team_WORK_OF_ART": {'candidates': []},
 'Barcelona_ORG': {'candidates': []},
 'La Liga_ORG': {'candidates': []},
 'Copa del Rey_EVENT': {'candidates': []},
 'UEFA Champions Leagues_ORG': {'candidates': []},
 'European_NORP': {'candidates': []},
 ' UEFA Champions League_ORG': {'candidates': []},
 ' Copa América_EVENT': {'candidates': []},
 'South American_NORP': {'candidates': []},
 'Spa_GPE': {'candidates': []},
 "Ballon d'Or_WORK_OF_ART": {'candidates': []},
 "Ballons d'Or_WORK_OF_ART": {'candidates': []},
 " Ballon d'Or behd_WORK_OF_ART": {'can

In [43]:
len(entity_and_link)

101

In [47]:
import time
st = time.time()
for e, grp in zip(selected_entities['word'], selected_entities['entity_group']):
    # lookup_results = EL_DBpedia_lookup(e, max_results=2)
    ky = e+"_"+grp
    if ky in entity_and_link and len(entity_and_link[ky]['candidates'])!=0:
        continue
    lookup_results = EL_DBpedia_lookup(e, max_results=2)
    entity_and_link[ky]['candidates'].extend(lookup_results)
en = time.time()
print(f"Time taken for lookup of {len(selected_entities)} entities is {en-st} seconds")

Time taken for lookup of 164 entities is 121.90965557098389 seconds


In [48]:
len(selected_entities['word'].unique()), len(selected_entities), len(entity_and_link)

(100, 164, 101)

In [49]:
entity_and_link

{'Lionel Andrés Messi_PERSON': {'candidates': ['http://dbpedia.org/resource/Lionel_Messi',
   'http://dbpedia.org/resource/Albacete_Balompié']},
 'Spanh_NORP': {'candidates': ['http://dbpedia.org/resource/Space_rock',
   'http://dbpedia.org/resource/NASA']},
 'Leo Messi_PERSON': {'candidates': ['http://dbpedia.org/resource/Lionel_Messi',
   'http://dbpedia.org/resource/List_of_minor_planets:_6001–7000']},
 'Argente_NORP': {'candidates': ['http://dbpedia.org/resource/Argentina',
   'http://dbpedia.org/resource/Argente']},
 'Ligue 1_ORG': {'candidates': ['http://dbpedia.org/resource/Algerian_Ligue_Professionnelle_1',
   'http://dbpedia.org/resource/Ligue_1']},
 'Par Sat-Germa_ORG': {'candidates': ['http://dbpedia.org/resource/List_of_K-1_events',
   'http://dbpedia.org/resource/United_Premier_Soccer_League']},
 'Argenta_GPE': {'candidates': ['http://dbpedia.org/resource/Argentina',
   'http://dbpedia.org/resource/Argentina_national_football_team']},
 'Messi_PERSON': {'candidates': ['http

In [52]:
import pickle
with open("./only_lookup_linking.pkl","wb") as f:
    pickle.dump(entity_and_link, f)

In [24]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [51]:
doc = nlp("England won the cricket world cup. England is a nation in Europe")
for ent in doc.ents:
    print(ent, ent.label_)

England GPE
England GPE
Europe LOC


In [53]:
%autoreload
from EntityLinking.ELmethods import (
    EL_DBpedia_lookup, 
    EL_DBpedia_spotlight,
    EL_redis_db
    )

In [54]:
for e in entity_and_link:
    _, uris = EL_DBpedia_spotlight(e.split("_")[0], nlp)
    for u in uris:
        link = uris[u][0]
        if u in entity_and_link:
            entity_and_link[e]['candidates'].append(link)


In [60]:
with open("./lookup_and_spotlight_linking.pkl","wb") as f:
    pickle.dump(entity_and_link, f)